In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_horizVert_wavelet_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,1618751.89,-22.92,-16.81,-11.28,-1.52,-0.99,0.27,-0.21,-0.00
3,-22.92,386051.61,-5.44,1.86,-0.02,-0.23,-0.11,0.03,0.02
4,-16.81,-5.44,94561.91,0.37,1.04,-0.04,-0.12,-0.01,-0.00
5,-11.28,1.86,0.37,19954.19,-0.04,-0.06,-0.02,-0.02,-0.01
6,-1.52,-0.02,1.04,-0.04,4528.93,0.04,0.01,0.00,0.00
7,-0.99,-0.23,-0.04,-0.06,0.04,1004.73,0.01,0.01,-0.00
8,0.27,-0.11,-0.12,-0.02,0.01,0.01,173.13,0.00,-0.00
9,-0.21,0.03,-0.01,-0.02,0.00,0.01,0.00,28.77,-0.00
10,-0.00,0.02,-0.00,-0.01,0.00,-0.00,-0.00,-0.00,2.16


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00003,-0.00004,-0.00006,-0.00002,-0.00002,0.00002,-0.00003,-0.00000
3,-0.00003,1.00000,-0.00003,0.00002,-0.00000,-0.00001,-0.00001,0.00001,0.00002
4,-0.00004,-0.00003,1.00000,0.00001,0.00005,-0.00000,-0.00003,-0.00000,-0.00000
5,-0.00006,0.00002,0.00001,1.00000,-0.00000,-0.00001,-0.00001,-0.00003,-0.00003
6,-0.00002,-0.00000,0.00005,-0.00000,1.00000,0.00002,0.00001,0.00001,0.00001
7,-0.00002,-0.00001,-0.00000,-0.00001,0.00002,1.00000,0.00001,0.00005,-0.00000
8,0.00002,-0.00001,-0.00003,-0.00001,0.00001,0.00001,1.00000,0.00002,-0.00001
9,-0.00003,0.00001,-0.00000,-0.00003,0.00001,0.00005,0.00002,1.00000,-0.00003
10,-0.00000,0.00002,-0.00000,-0.00003,0.00001,-0.00000,-0.00001,-0.00003,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

44.11432675481947

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.91443713e+11 1.63078858e+10 9.58064630e+08 4.15147423e+07
 2.05339808e+06 9.46539719e+04 2.50687709e+03 5.16616523e+01
 6.02290843e-31]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999474,-0.031562,-0.007271,-0.001534,-0.000347,-0.000077,-0.000013,-0.000002,-1.664411e-07
1,0.031256,0.998782,-0.037411,-0.007367,-0.001669,-0.000371,-0.000064,-0.000011,-7.522463e-07
2,0.008369,0.036854,0.998566,-0.037037,-0.007980,-0.001769,-0.000306,-0.000051,-3.807480e-06
3,0.002049,0.008573,0.036284,0.998096,-0.048022,-0.010029,-0.001724,-0.000287,-2.174744e-05
4,0.000557,0.002329,0.009514,0.047000,0.997081,-0.058583,-0.009492,-0.001574,-1.177610e-04
5,0.000155,0.000649,0.002645,0.012564,0.057345,0.996448,-0.059547,-0.009508,-7.127449e-04
6,0.000035,0.000148,0.000604,0.002857,0.012592,0.057855,0.994575,-0.085255,-6.130332e-03
7,0.000009,0.000037,0.000152,0.000719,0.003159,0.014244,0.083826,0.993503,-7.557106e-02
8,0.000001,0.000006,0.000023,0.000108,0.000475,0.002140,0.012424,0.074766,9.971213e-01


In [ ]:
cos_dist = spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine')
cos_dist = np.sort(np.concatenate(cos_dist))[:len(GROUPS)]
cos_dist


array([0.00052589, 0.00121762, 0.00143449, 0.00190378, 0.00287869,
       0.00291919, 0.00355154, 0.00542483, 0.00649673])